In [ ]:
import torch


In [4]:
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

True
NVIDIA GeForce RTX 3050 Laptop GPU


In [1]:
#import libraries
import torch,torchvision,pdb,os
from torch import nn
from  torch.utils.data import DataLoader,Dataset
from torchvision import transforms
from torchvision.utils import make_grid
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from tqdm.auto import tqdm
import wandb

c:\Users\tejag\OneDrive\Desktop\face\face-image-generator-\naathing\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def show_from_tensor(tensor,num=25,wandb_log=1,name=" "):
    img=tensor.detach().cpu()
    images=make_grid(img[:num],nrow=5).permute(1,2,0)
    if (wandb_log==1 and wandb_active==1):
        wandb.log({name:wandb.Image(images.numpy().clip(0,1))})
    
    plt.figure(figsize=(10,10))
    plt.imshow(images.clip(0,1))
    plt.show()

In [3]:
#defining hyper parameters
batch_size=128
epoch=1000
wandb_active=1
critic_cycle=5
lr=0.00001
z_dim=200
mean_gen_loss=[]
mean_critic_loss=[]
curr_step=0
device='cuda'
show_step=35
save_step=35

In [4]:
wandb.login(key="7b043259eca153894ca3a1f456f024961f4a979d")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\tejag\_netrc
wandb: Currently logged in as: gunateja951 (gunateja951-omni) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [5]:
experiment_name="face-image-generator"
wandb.init(
    project="test",
    group=experiment_name,
    config={
        "batch_size": batch_size,
        "epoch": epoch,
        "critic_cycle": critic_cycle,
        "lr": lr,
        "z_dim": z_dim,
        "mean_gen_loss": mean_gen_loss,
        "mean_critic_loss": mean_critic_loss,
        "curr_step": curr_step,
        "device": device,
        "show_step": show_step,
        "save_step": save_step
    }
)
config= wandb.config

In [ ]:
class Generator(nn.Module):
    def __init__(self,z_dim=200,i_dim=16):
        super(Generator,self).__init__()
        self.z_dim=z_dim
        self.i_dim=i_dim
        self.gen=nn.Sequential(
            nn.ConvTranspose2d(z_dim,i_dim*16,4,1,0),
            nn.BatchNorm2d(i_dim*16),
            nn.ReLU(inplace=True),
            
            nn.ConvTranspose2d(i_dim*16,i_dim*8,4,2,1),
            nn.BatchNorm2d(i_dim*8),
            nn.ReLU(inplace=True),
            
            nn.ConvTranspose2d(i_dim*8,i_dim*4,4,2,1),
            nn.BatchNorm2d(i_dim*4),
            nn.ReLU(inplace=True),
            
            nn.ConvTranspose2d(i_dim*4,i_dim*2,4,2,1),
            nn.BatchNorm2d(i_dim*2),
            nn.ReLU(inplace=True),
            
            nn.ConvTranspose2d(i_dim*2,3,4,2,1),
            nn.Tanh()
        )
    
    def forward(self,noise):
        x=noise.view(len(noise),self.z_dim,1,1)
        return self.gen(x)
        
def get_noise(n_samples,z_dim):
    return torch.randn(n_samples,z_dim).to(device)

In [ ]:
class Critic(nn.Module):
    def __init__(self,i_dim=16):
        super(Critic,self).__init__()
        self.i_dim=i_dim
        self.critic=nn.Sequential(
            nn.Conv2d(3,i_dim*2,4,2,1),
            nn.InstanceNorm2d(i_dim*2),
            nn.LeakyReLU(0.2),
            
            nn.Conv2d(i_dim*2,i_dim*4,4,2,1),
            nn.InstanceNorm2d(i_dim*4),
            nn.LeakyReLU(0.2),
            
            nn.Conv2d(i_dim*4,i_dim*8,4,2,1),
            nn.InstanceNorm2d(i_dim*8),
            nn.LeakyReLU(0.2),
            
            nn.Conv2d(i_dim*8,i_dim*16,4,2,1),
            nn.InstanceNorm2d(i_dim*16),
            nn.LeakyReLU(0.2),
            
            nn.Conv2d(i_dim*16,1,4,1,0),
        )
    
    def forward(self,x):
        x=self.critic(x)
        return x.view(len(x),-1)

In [9]:
import gdown
import os

file_id = '1rnrH6dDlgj8aRRxgGcznCGKlXAUiUAC5'
output_path = 'data/img_align_celeba.7z'

os.makedirs('data', exist_ok=True)

# Download your private Drive file
gdown.download(f'https://drive.google.com/uc?id={file_id}', output_path, quiet=False)


Downloading...
From (original): https://drive.google.com/uc?id=1rnrH6dDlgj8aRRxgGcznCGKlXAUiUAC5
From (redirected): https://drive.google.com/uc?id=1rnrH6dDlgj8aRRxgGcznCGKlXAUiUAC5&confirm=t&uuid=eee82fa8-30db-47b4-bb0a-abdca3b00f44
To: c:\Users\tejag\OneDrive\Desktop\face\face-image-generator-\data\img_align_celeba.7z
100%|██████████| 1.44G/1.44G [07:35<00:00, 3.17MB/s]


'data/img_align_celeba.7z'

In [23]:
import subprocess
import time

archive_path = r"C:\Users\tejag\OneDrive\Desktop\face\face-image-generator-\data\img_align_celeba.7z"
output_path = r"C:\Users\tejag\OneDrive\Desktop\face\face-image-generator-\data\img_align_celeba"

start = time.time()

subprocess.run([
    r"C:\Program Files\7-Zip\7z.exe",
    'x',
    archive_path,
    f'-o{output_path}',
    '-mmt=on'
])

print(f"Extraction completed in {time.time() - start:.2f} seconds")


Extraction completed in 487.50 seconds


In [29]:
class dataset(Dataset):
    def __init__(self,path,lim=1000,size=128):
        self.size=[size,size]
        self.lim=lim
        index,value=[],[]
        for data in os.listdir(path)[:lim]:
            item=os.path.join(path,data)
            value.append(item)
            index.append(data)
        self.index=index
        self.value=value
        
    def __len__(self):
        return len(self.value)
    
    def __getitem__(self,idx):
        img=Image.open(self.value[idx]).convert("RGB")
        img=np.asarray(torchvision.transforms.Resize(self.size)(img))
        img=np.transpose(img,(2,0,1)).astype(np.float32)
        img=torch.from_numpy(img).mul(255)
        return img,self.index[idx]
dataset=dataset("data/img_align_celeba",lim=1000,size=128)
ds=DataLoader(dataset,batch_size=batch_size,shuffle=True,num_workers=0)

gen=Generator(z_dim).to(device)
cri=Critic().to(device)

gen_opt=torch.optim.Adam(gen.parameters(),lr=lr,betas=(0.5,0.9))
critic_opt=torch.optim.Adam(cri.parameters(),lr=lr,betas=(0.5,0.9))

if(wandb_active==1):
    wandb.watch(gen,log_freq=100)
    wandb.watch(cri,log_freq=100)

x,y=next(iter(ds))


show_from_tensor(x)
        

Error in callback <bound method _WandbInit._pre_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x00000222E4E54FD0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 222e741aa30, raw_cell="class dataset(Dataset):
    def __init__(self,path.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/tejag/OneDrive/Desktop/face/face-image-generator-/model.ipynb#X23sZmlsZQ%3D%3D>,),kwargs {}:


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

TypeError: Generator.__init__() takes 1 positional argument but 2 were given

Error in callback <bound method _WandbInit._post_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x00000222E4E54FD0>> (for post_run_cell), with arguments args (<ExecutionResult object at 222e7a2e1f0, execution_count=29 error_before_exec=None error_in_exec=Generator.__init__() takes 1 positional argument but 2 were given info=<ExecutionInfo object at 222e741aa30, raw_cell="class dataset(Dataset):
    def __init__(self,path.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/tejag/OneDrive/Desktop/face/face-image-generator-/model.ipynb#X23sZmlsZQ%3D%3D> result=None>,),kwargs {}:


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host